<a href="https://colab.research.google.com/github/Joshua-Dias-Barreto/runtimeTerror_ML_Hackathon_PS2/blob/main/runtimeTerror_PS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from google.colab import files
from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input

# Input Data and Preprocessing

In [ ]:
df = pd.read_csv('ageImgDf.csv')
X = np.load('ageImg.npy')
X=X/255
y = np.array(df['age'])
X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                   test_size=0.25,
                                   shuffle=True)


# Call backs

In [ ]:
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from timeit import default_timer as timer

class TimingCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

early_stopping = EarlyStopping(
                                patience=5,
                                min_delta = 0.01,

                                verbose=1,
                                mode = 'min',
                                monitor='val_loss')

reduce_learning_rate = ReduceLROnPlateau(
                                    monitor="val_loss",
                                    patience=3,
                                    episilon= 0.01,
                                    factor=0.1,
                                    cooldown = 4,
                                    verbose=1)

time_callback = TimingCallback()


lr = 0.01
epochs = 30
batch_size = 128
results = {}
num_of_ages = 95

# Model

In [ ]:
from keras.applications.mobilenet import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras.models import Sequential
base_mobilenet_model = MobileNet(input_shape =  (128,128, 1),
                                 include_top = False,
                                 weights = None)
mobilenet_model = Sequential()
mobilenet_model.add(BatchNormalization(input_shape = (128,128, 1)))
mobilenet_model.add(base_mobilenet_model)
mobilenet_model.add(BatchNormalization())
mobilenet_model.add(GlobalAveragePooling2D())
mobilenet_model.add(Dropout(0.5))
mobilenet_model.add(Dense(1,activation='linear'))


mobilenet_model.compile(optimizer = 'adam', loss = 'mean_absolute_error',
                           metrics = ["mean_absolute_error"])




# Training History

In [ ]:
mobilenet_model_history = mobilenet_model.fit(
            x=X_train, y=y_train,
            validation_split=0.2,
            epochs = epochs,
            callbacks = [
                         reduce_learning_rate,
                         early_stopping,
                         time_callback
                        ],
            verbose=True)

Epoch 1/30
183/183 [==============================] - 406s 2s/step - loss: 22.8907 - mean_absolute_error: 22.8907 - val_loss: 20.8371 - val_mean_absolute_error: 20.8371 - lr: 0.0010
Epoch 2/30
183/183 [==============================] - 360s 2s/step - loss: 11.5993 - mean_absolute_error: 11.5993 - val_loss: 21.4447 - val_mean_absolute_error: 21.4447 - lr: 0.0010
Epoch 3/30
183/183 [==============================] - 355s 2s/step - loss: 9.7817 - mean_absolute_error: 9.7817 - val_loss: 20.9158 - val_mean_absolute_error: 20.9158 - lr: 0.0010
Epoch 4/30
183/183 [==============================] - 356s 2s/step - loss: 9.1276 - mean_absolute_error: 9.1276 - val_loss: 11.0118 - val_mean_absolute_error: 11.0118 - lr: 0.0010
Epoch 5/30
183/183 [==============================] - 358s 2s/step - loss: 8.1987 - mean_absolute_error: 8.1987 - val_loss: 8.6694 - val_mean_absolute_error: 8.6694 - lr: 0.0010
Epoch 6/30
183/183 [==============================] - 355s 2s/step - loss: 7.8855 - mean_absolute_

# Performance on Test set

In [ ]:
mobilenet_model_test_results = mobilenet_model.evaluate(X_test, y_test, batch_size=128)
print(dict(zip(mobilenet_model.metrics_names, mobilenet_model_test_results)))

20/20 [==============================] - 31s 1s/step - loss: 6.2965 - mean_absolute_error: 6.2965
{'loss': 6.296520233154297, 'mean_absolute_error': 6.296520233154297}
